# Limpieza & Set Up de BBDD

### Realizaremos algunas customizaciones sobre la base con el fin de optimizar el trabajo de aqui en adelante:

    0.  Importar Librerias & Lectura de mi dt
    1.  Cambiar los títulos de las columnas a mayúsculas
    2.  Agregar la leyenda del índice
    3.  Conversión de columnas, Time & AvgTalkDuration
    4.  Crear  columna "DAY_OF_THE_WEEK"
    5.  Limpiaza & generación de embudo
    6.  Verificar que no existen nulo en la Base de Datos
    7.  Verificar que no existan duplicados
    8.  Verificar que no existan absurdos
    9.  Ordenar las columnas
    10. Guardar un CSV con la base de datos limpia y seteada para iniciar nuestro analisis
    

# 0. Importación de librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Lectura de mi base de datos en  bruto

In [2]:
rutaexcel = r"C:\\Users\\Lucia\Desktop\\bootcamp\\Proyecto_EDA_Pablo_Mendez\\Telecom Company Call-Center-Dataset.xlsx"
bbdd = pd.read_excel(rutaexcel, parse_dates=['Date'],
                     dtype={'Speed of answer in seconds': float,
                            'Satisfaction rating': float})
bbdd

,Call Id,Agent,Date,Time,Topic,Answered (Y/N),Resolved,Speed of answer in seconds,AvgTalkDuration,Satisfaction rating
0,ID0033,Diane,2021-01-01,13:33:36,Technical Support,Y,Y,57.0,00:06:49,3.0
1,ID0056,Dan,2021-01-01,17:44:10,Streaming,Y,Y,83.0,00:06:45,4.0
2,ID0039,Stewart,2021-01-01,14:38:24,Contract related,Y,Y,74.0,00:06:41,1.0
3,ID0009,Greg,2021-01-01,11:13:55,Admin Support,Y,Y,15.0,00:06:38,4.0
4,ID0043,Diane,2021-01-01,14:47:02,Technical Support,Y,Y,45.0,00:06:21,3.0
...,...,...,...,...,...,...,...,...,...,...
4995,ID4992,Joe,2021-03-31,14:13:55,Streaming,N,N,NaN,NaN,NaN
4996,ID4993,Joe,2021-03-31,14:18:14,Admin Support,N,N,NaN,NaN,NaN
4997,ID4994,Greg,2021-03-31,14:58:34,Contract related,N,N,NaN,NaN,NaN
4998,ID4995,Dan,2021-03-31,15:33:07,Payment related,N,N,NaN,NaN,NaN


In [3]:
bbdd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Call Id                     5000 non-null   object        
 1   Agent                       5000 non-null   object        
 2   Date                        5000 non-null   datetime64[ns]
 3   Time                        5000 non-null   object        
 4   Topic                       5000 non-null   object        
 5   Answered (Y/N)              5000 non-null   object        
 6   Resolved                    5000 non-null   object        
 7   Speed of answer in seconds  4054 non-null   float64       
 8   AvgTalkDuration             4054 non-null   object        
 9   Satisfaction rating         4054 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 390.8+ KB


#     1. Cambiar los títulos de las columnas a mayúsculas 
#     2. Agregar la leyenda del índice
###      Para trabajar de una manera mas ordenada se decide insertar un indice al cual se puede llamar en el futuro


In [4]:
bbdd.columns = bbdd.columns.str.upper()
bbdd
bbdd.index.name = 'INDEX'

In [5]:
bbdd

,CALL ID,AGENT,DATE,TIME,TOPIC,ANSWERED (Y/N),RESOLVED,SPEED OF ANSWER IN SECONDS,AVGTALKDURATION,SATISFACTION RATING
INDEX,,,,,,,,,,
0,ID0033,Diane,2021-01-01,13:33:36,Technical Support,Y,Y,57.0,00:06:49,3.0
1,ID0056,Dan,2021-01-01,17:44:10,Streaming,Y,Y,83.0,00:06:45,4.0
2,ID0039,Stewart,2021-01-01,14:38:24,Contract related,Y,Y,74.0,00:06:41,1.0
3,ID0009,Greg,2021-01-01,11:13:55,Admin Support,Y,Y,15.0,00:06:38,4.0
4,ID0043,Diane,2021-01-01,14:47:02,Technical Support,Y,Y,45.0,00:06:21,3.0
...,...,...,...,...,...,...,...,...,...,...
4995,ID4992,Joe,2021-03-31,14:13:55,Streaming,N,N,NaN,NaN,NaN
4996,ID4993,Joe,2021-03-31,14:18:14,Admin Support,N,N,NaN,NaN,NaN
4997,ID4994,Greg,2021-03-31,14:58:34,Contract related,N,N,NaN,NaN,NaN


# 3.Conversión de columnas, Time & AvgTalkDuration

### La comuna 'Time' se modificara como tipo de dato a delta time
### La columna 'AvgTalkDuration' se modificara como tipo de dato datetime con el formato dias, horas, minutos y segundos.


In [6]:
bbdd['TIME'] = pd.to_datetime(bbdd['TIME'], format='%H:%M:%S')

bbdd['TIME'] = bbdd['TIME'].dt.hour * pd.Timedelta(hours=1) + bbdd['TIME'].dt.minute * pd.Timedelta(minutes=1) + bbdd['TIME'].dt.second * pd.Timedelta(seconds=1)


In [33]:
bbdd

,CALL ID,AGENT,DATE,DAY_OF_THE_WEEK,TIME,TOPIC,ANSWERED (Y/N),RESOLVED,SPEED OF ANSWER IN SECONDS,AVGTALKDURATION,SATISFACTION RATING
INDEX,,,,,,,,,,,
0,ID0033,Diane,2021-01-01,Friday,13,Technical Support,Y,Y,57.0,0 days 00:06:49,3.0
1,ID0056,Dan,2021-01-01,Friday,17,Streaming,Y,Y,83.0,0 days 00:06:45,4.0
2,ID0039,Stewart,2021-01-01,Friday,14,Contract related,Y,Y,74.0,0 days 00:06:41,1.0
3,ID0009,Greg,2021-01-01,Friday,11,Admin Support,Y,Y,15.0,0 days 00:06:38,4.0
4,ID0043,Diane,2021-01-01,Friday,14,Technical Support,Y,Y,45.0,0 days 00:06:21,3.0
...,...,...,...,...,...,...,...,...,...,...,...
4995,ID4992,Joe,2021-03-31,Wednesday,14,Streaming,N,N,NaN,0 days 00:00:00,NaN
4996,ID4993,Joe,2021-03-31,Wednesday,14,Admin Support,N,N,NaN,0 days 00:00:00,NaN
4997,ID4994,Greg,2021-03-31,Wednesday,14,Contract related,N,N,NaN,0 days 00:00:00,NaN


#### Unicamente se completan los Nan de la columna con el fin de convetir el tipo de dato proximamente se analizaran por partes el Dt con el fin de analizarlo en 2 etapas del "embudo de analisis" 

In [8]:
import pandas as pd
from datetime import datetime, timedelta

bbdd['AVGTALKDURATION'].fillna('00:00:00', inplace=True)

bbdd['AVGTALKDURATION'] = pd.to_datetime(bbdd['AVGTALKDURATION'], format='%H:%M:%S').dt.time


bbdd['AVGTALKDURATION'] = bbdd['AVGTALKDURATION'].apply(lambda x: timedelta(hours=x.hour, minutes=x.minute, seconds=x.second))

In [9]:
bbdd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   CALL ID                     5000 non-null   object         
 1   AGENT                       5000 non-null   object         
 2   DATE                        5000 non-null   datetime64[ns] 
 3   TIME                        5000 non-null   datetime64[ns] 
 4   TOPIC                       5000 non-null   object         
 5   ANSWERED (Y/N)              5000 non-null   object         
 6   RESOLVED                    5000 non-null   object         
 7   SPEED OF ANSWER IN SECONDS  4054 non-null   float64        
 8   AVGTALKDURATION             5000 non-null   timedelta64[ns]
 9   SATISFACTION RATING         4054 non-null   float64        
dtypes: datetime64[ns](2), float64(2), object(5), timedelta64[ns](1)
memory usage: 390.8+ KB


In [10]:
bbdd['AVGTALKDURATION'].mean()

Timedelta('0 days 00:03:02.367400')

#     4. Creación de columna "DAY_OF_THE_WEEK"
###      Se decide crear esta columna para visualizar de una manera mas simple los días en que la empresa brinda servivio. 

In [11]:
bbdd['DAY_OF_THE_WEEK'] = pd.to_datetime(bbdd['DATE']).dt.day_name()
bbdd


,CALL ID,AGENT,DATE,TIME,TOPIC,ANSWERED (Y/N),RESOLVED,SPEED OF ANSWER IN SECONDS,AVGTALKDURATION,SATISFACTION RATING,DAY_OF_THE_WEEK
INDEX,,,,,,,,,,,
0,ID0033,Diane,2021-01-01,1900-01-01 13:33:36,Technical Support,Y,Y,57.0,0 days 00:06:49,3.0,Friday
1,ID0056,Dan,2021-01-01,1900-01-01 17:44:10,Streaming,Y,Y,83.0,0 days 00:06:45,4.0,Friday
2,ID0039,Stewart,2021-01-01,1900-01-01 14:38:24,Contract related,Y,Y,74.0,0 days 00:06:41,1.0,Friday
3,ID0009,Greg,2021-01-01,1900-01-01 11:13:55,Admin Support,Y,Y,15.0,0 days 00:06:38,4.0,Friday
4,ID0043,Diane,2021-01-01,1900-01-01 14:47:02,Technical Support,Y,Y,45.0,0 days 00:06:21,3.0,Friday
...,...,...,...,...,...,...,...,...,...,...,...
4995,ID4992,Joe,2021-03-31,1900-01-01 14:13:55,Streaming,N,N,NaN,0 days 00:00:00,NaN,Wednesday
4996,ID4993,Joe,2021-03-31,1900-01-01 14:18:14,Admin Support,N,N,NaN,0 days 00:00:00,NaN,Wednesday
4997,ID4994,Greg,2021-03-31,1900-01-01 14:58:34,Contract related,N,N,NaN,0 days 00:00:00,NaN,Wednesday


##### Podemos inferir que se brinda servicio los 7 días de la semana

In [12]:
pd.pivot_table(bbdd, values=['CALL ID', 'AGENT', 'DATE', 'TIME', 'TOPIC', 'ANSWERED (Y/N)', 'RESOLVED', 'SPEED OF ANSWER IN SECONDS', 'AVGTALKDURATION', 'SATISFACTION RATING'], index='DAY_OF_THE_WEEK', aggfunc='first')

,AGENT,ANSWERED (Y/N),AVGTALKDURATION,CALL ID,DATE,RESOLVED,SATISFACTION RATING,SPEED OF ANSWER IN SECONDS,TIME,TOPIC
DAY_OF_THE_WEEK,,,,,,,,,,
Friday,Diane,Y,0 days 00:06:49,ID0033,2021-01-01,Y,3.0,57.0,1900-01-01 13:33:36,Technical Support
Monday,Stewart,Y,0 days 00:06:50,ID0219,2021-01-04,Y,5.0,120.0,1900-01-01 15:48:58,Admin Support
Saturday,Greg,Y,0 days 00:06:56,ID0082,2021-01-02,Y,4.0,52.0,1900-01-01 12:00:00,Payment related
Sunday,Martha,Y,0 days 00:06:59,ID0153,2021-01-03,N,4.0,52.0,1900-01-01 12:54:43,Payment related
Thursday,Stewart,Y,0 days 00:06:50,ID0362,2021-01-07,Y,3.0,66.0,1900-01-01 11:42:43,Technical Support
Tuesday,Diane,Y,0 days 00:06:48,ID0264,2021-01-05,Y,3.0,72.0,1900-01-01 15:31:41,Streaming
Wednesday,Stewart,Y,0 days 00:06:54,ID0336,2021-01-06,Y,5.0,37.0,1900-01-01 17:00:58,Contract related


In [13]:
bbdd['DAY_OF_THE_WEEK'].unique()

array(['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'], dtype=object)

#     5.  Limpiaza & generación de embudo para analisis de datos


#### En primer lugar verificamos que no figuren datos existentes en las llamadas concretadas

In [14]:
interacciones_no_concretadas = bbdd[bbdd['ANSWERED (Y/N)'] == 'N']
interacciones_no_concretadas[interacciones_no_concretadas['AVGTALKDURATION']!='0 days 00:00:00']


,CALL ID,AGENT,DATE,TIME,TOPIC,ANSWERED (Y/N),RESOLVED,SPEED OF ANSWER IN SECONDS,AVGTALKDURATION,SATISFACTION RATING,DAY_OF_THE_WEEK
INDEX,,,,,,,,,,,


 ###  Una vez corroborado, procederemos separar los dt con el fin de analizaros por 2 partes distintas del embudo por un lado las interaciones concretadas y por otro lado las interaciones concretadas 

In [15]:
bbdd_Y = bbdd[bbdd['ANSWERED (Y/N)'] == 'Y']
bbdd_N = bbdd[bbdd['ANSWERED (Y/N)'] == 'N']


In [16]:
bbdd_Y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4054 entries, 0 to 4992
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   CALL ID                     4054 non-null   object         
 1   AGENT                       4054 non-null   object         
 2   DATE                        4054 non-null   datetime64[ns] 
 3   TIME                        4054 non-null   datetime64[ns] 
 4   TOPIC                       4054 non-null   object         
 5   ANSWERED (Y/N)              4054 non-null   object         
 6   RESOLVED                    4054 non-null   object         
 7   SPEED OF ANSWER IN SECONDS  4054 non-null   float64        
 8   AVGTALKDURATION             4054 non-null   timedelta64[ns]
 9   SATISFACTION RATING         4054 non-null   float64        
 10  DAY_OF_THE_WEEK             4054 non-null   object         
dtypes: datetime64[ns](2), float64(2), object(6), tim

In [17]:
bbdd['SATISFACTION RATING'].mean()

3.4035520473606313

# 6 Verificación de Nulos en la Base de Datos

In [18]:
bbdd_Y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4054 entries, 0 to 4992
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   CALL ID                     4054 non-null   object         
 1   AGENT                       4054 non-null   object         
 2   DATE                        4054 non-null   datetime64[ns] 
 3   TIME                        4054 non-null   datetime64[ns] 
 4   TOPIC                       4054 non-null   object         
 5   ANSWERED (Y/N)              4054 non-null   object         
 6   RESOLVED                    4054 non-null   object         
 7   SPEED OF ANSWER IN SECONDS  4054 non-null   float64        
 8   AVGTALKDURATION             4054 non-null   timedelta64[ns]
 9   SATISFACTION RATING         4054 non-null   float64        
 10  DAY_OF_THE_WEEK             4054 non-null   object         
dtypes: datetime64[ns](2), float64(2), object(6), tim

#### Verificación de Nulos y NaN

In [19]:
bbdd_Y.isna()

,CALL ID,AGENT,DATE,TIME,TOPIC,ANSWERED (Y/N),RESOLVED,SPEED OF ANSWER IN SECONDS,AVGTALKDURATION,SATISFACTION RATING,DAY_OF_THE_WEEK
INDEX,,,,,,,,,,,
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
4988,False,False,False,False,False,False,False,False,False,False,False
4989,False,False,False,False,False,False,False,False,False,False,False
4990,False,False,False,False,False,False,False,False,False,False,False


In [20]:
bbdd_Y.isnull()

,CALL ID,AGENT,DATE,TIME,TOPIC,ANSWERED (Y/N),RESOLVED,SPEED OF ANSWER IN SECONDS,AVGTALKDURATION,SATISFACTION RATING,DAY_OF_THE_WEEK
INDEX,,,,,,,,,,,
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
4988,False,False,False,False,False,False,False,False,False,False,False
4989,False,False,False,False,False,False,False,False,False,False,False
4990,False,False,False,False,False,False,False,False,False,False,False


# 7 Verificación de duplicados

 #### inicialmente corroboramos si hay algun duplicado a nivel de la base en general y luego ponemos foco en la columna CALL ID (es la unica que no deberia tener duplicados)

In [21]:
bbdd_Y[bbdd_Y.duplicated()].shape[0]

0

In [22]:
bbdd_Y[bbdd_Y['CALL ID'].duplicated()].shape[0]

0

# 8 Verificación de absurdos
### Recurimos a analizar los valores unicos de cada columna con el fin de visualizar si algun dato fuera absurdo

Verificar que no existan absurdos con mayores de  XX 

In [23]:
def valores_unicos(bbdd_Y):
    detalle = {}
    for columna in bbdd.columns:
        valores_unicos = bbdd_Y[columna].unique()
        detalle[columna] = valores_unicos
    return detalle    

In [24]:
detalle_de_valores_unicos = valores_unicos(bbdd_Y)
detalle_de_valores_unicos

{'CALL ID': array(['ID0033', 'ID0056', 'ID0039', ..., 'ID4998', 'ID4999', 'ID4978'],
       dtype=object),
 'AGENT': array(['Diane', 'Dan', 'Stewart', 'Greg', 'Martha', 'Becky', 'Joe', 'Jim'],
       dtype=object),
 'DATE': <DatetimeArray>
 ['2021-01-01 00:00:00', '2021-01-02 00:00:00', '2021-01-03 00:00:00',
  '2021-01-04 00:00:00', '2021-01-05 00:00:00', '2021-01-06 00:00:00',
  '2021-01-07 00:00:00', '2021-01-08 00:00:00', '2021-01-09 00:00:00',
  '2021-01-10 00:00:00', '2021-01-11 00:00:00', '2021-01-12 00:00:00',
  '2021-01-13 00:00:00', '2021-01-14 00:00:00', '2021-01-15 00:00:00',
  '2021-01-16 00:00:00', '2021-01-17 00:00:00', '2021-01-18 00:00:00',
  '2021-01-19 00:00:00', '2021-01-20 00:00:00', '2021-01-21 00:00:00',
  '2021-01-22 00:00:00', '2021-01-23 00:00:00', '2021-01-24 00:00:00',
  '2021-01-25 00:00:00', '2021-01-26 00:00:00', '2021-01-27 00:00:00',
  '2021-01-28 00:00:00', '2021-01-29 00:00:00', '2021-01-30 00:00:00',
  '2021-01-31 00:00:00', '2021-02-01 00:00:00', '2

#### Analizamos absurdos en la columna TIME con el fin de visualizar si existe alguna llamada fuera del horario de atención

In [25]:
bbdd_Y.index = pd.to_datetime(bbdd_Y.index)

Absurdos_TIME = bbdd_Y['TIME'].between_time('18:00:00', '23:59:00') | (bbdd_Y['TIME'].between_time('00:00:00', '09:00:00'))
print(Absurdos_TIME)

INDEX
1970-01-01 00:00:00.000000000    False
1970-01-01 00:00:00.000000001    False
1970-01-01 00:00:00.000000002    False
1970-01-01 00:00:00.000000003    False
1970-01-01 00:00:00.000000004    False
                                 ...  
1970-01-01 00:00:00.000004988    False
1970-01-01 00:00:00.000004989    False
1970-01-01 00:00:00.000004990    False
1970-01-01 00:00:00.000004991    False
1970-01-01 00:00:00.000004992    False
Name: TIME, Length: 4054, dtype: bool


C:\Users\Lucia\AppData\Local\Temp\ipykernel_19304\1592723899.py:3: FutureWarning: Operation between non boolean Series with different indexes will no longer return a boolean result in a future version. Cast both Series to object type to maintain the prior behavior.
  Absurdos_TIME = bbdd_Y['TIME'].between_time('18:00:00', '23:59:00') | (bbdd_Y['TIME'].between_time('00:00:00', '09:00:00'))


In [26]:
bbdd_Y.reset_index(drop=True, inplace=True)

#  9 Ordenamos las columnas par, con el fin de agilizar la visualización de a que dia corresponde la fecha

In [27]:
bbdd_Y = bbdd_Y[['CALL ID', 'AGENT', 'DATE','DAY_OF_THE_WEEK', 'TIME', 'TOPIC', 'ANSWERED (Y/N)',
       'RESOLVED', 'SPEED OF ANSWER IN SECONDS', 'AVGTALKDURATION',
       'SATISFACTION RATING']]

In [28]:
bbdd_N = bbdd_N[['CALL ID', 'AGENT', 'DATE','DAY_OF_THE_WEEK', 'TIME', 'TOPIC', 'ANSWERED (Y/N)',
       'RESOLVED', 'SPEED OF ANSWER IN SECONDS', 'AVGTALKDURATION',
       'SATISFACTION RATING']]

In [29]:
bbdd = bbdd[['CALL ID', 'AGENT', 'DATE','DAY_OF_THE_WEEK', 'TIME', 'TOPIC', 'ANSWERED (Y/N)',
       'RESOLVED', 'SPEED OF ANSWER IN SECONDS', 'AVGTALKDURATION',
       'SATISFACTION RATING']]

In [30]:
bbdd_Y

,CALL ID,AGENT,DATE,DAY_OF_THE_WEEK,TIME,TOPIC,ANSWERED (Y/N),RESOLVED,SPEED OF ANSWER IN SECONDS,AVGTALKDURATION,SATISFACTION RATING
0,ID0033,Diane,2021-01-01,Friday,1900-01-01 13:33:36,Technical Support,Y,Y,57.0,0 days 00:06:49,3.0
1,ID0056,Dan,2021-01-01,Friday,1900-01-01 17:44:10,Streaming,Y,Y,83.0,0 days 00:06:45,4.0
2,ID0039,Stewart,2021-01-01,Friday,1900-01-01 14:38:24,Contract related,Y,Y,74.0,0 days 00:06:41,1.0
3,ID0009,Greg,2021-01-01,Friday,1900-01-01 11:13:55,Admin Support,Y,Y,15.0,0 days 00:06:38,4.0
4,ID0043,Diane,2021-01-01,Friday,1900-01-01 14:47:02,Technical Support,Y,Y,45.0,0 days 00:06:21,3.0
...,...,...,...,...,...,...,...,...,...,...,...
4049,ID4987,Stewart,2021-03-31,Wednesday,1900-01-01 13:01:55,Payment related,Y,N,10.0,0 days 00:02:26,4.0
4050,ID4986,Diane,2021-03-31,Wednesday,1900-01-01 12:50:24,Streaming,Y,N,55.0,0 days 00:02:01,1.0
4051,ID4998,Diane,2021-03-31,Wednesday,1900-01-01 16:53:46,Payment related,Y,Y,84.0,0 days 00:01:49,4.0
4052,ID4999,Jim,2021-03-31,Wednesday,1900-01-01 17:02:24,Streaming,Y,Y,98.0,0 days 00:00:58,5.0


# 10 Se procede a guardar las bases de datos depuradas en nuevos CSV con el fin de poder llamarla en nuestros futuros analisis y no tener que correr todos las customizaciones

In [31]:
bbdd_Y.to_csv('telecom_Y.csv', index=False)
bbdd_N.to_csv('telecom_N.csv', index=False)
bbdd.to_csv('telecom.csv', index=False)
